# getting presidential election exit poll results

This jupyter notebook is not used directly by the web application but this is how data is scraped from the website. 

## scraping exit poll data from CNN

In [34]:
#connecting to the url 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")
driver=webdriver.Chrome(options=options)

area = "ohio" #this is the area, national here, can be change to any valid state that is available on the website
url="https://www.cnn.com/election/2016/results/exit-polls/"+area+"/president"
driver.get(url)



C:\Users\maxji\AppData\Local\Temp\ipykernel_16340\449253272.py:10: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [35]:
#locating all divs that contain tables and the name of the table, according to the format of the website

all_tables = driver.find_element(By.XPATH, "//div[@class='exit-poll-list exit-poll-list--framed']")#setting the div that wraps all the tables
table_divs = all_tables.find_elements(By.XPATH, "//div[contains(@class, 'exit-poll-table exit-poll-table--framed')]") #finding the list of separate table divs from the container found above

In [28]:
print(len(table_divs))

68


In [36]:
#extracting the data by looping through the table divs

#make a new folder 
main_path = "2016_presidential_data"
!mkdir {main_path}
#categories to save csv data for

#create a new directory
for table_div in table_divs:
    #find current header or the table name
    header = str(table_div.find_element(By.TAG_NAME, "h3").text) #get current 
    #only save data that belongs to these categories
    categories = ["gender", "age", "race", "area type"]
    if header in categories:
        print("current header: ", header)
        columns = []#define dataframe columns which will be used to write to a csv file

        #the first row defines the columns of the csv file, with the first part being the 
        header_row = table_div.find_element(By.TAG_NAME, "thead")
        header_data = header_row.find_elements(By.TAG_NAME, "th") #data entries in header rows
        for i in range (len(header_data)): 
            #in these tables, the first entry in the header row is empty, so add content 
            if i == 0:
                columns.append("sample percentage and category")
            else: 
                current_text =str(header_data[i].text)
                #this particular datafile does not have the candicates capitalized, so do it
                current_text = current_text.replace("clinton", "Clinton")
                current_text = current_text.replace("trump", "Trump")
                columns.append("% for " + current_text)
        #find the body rows
        #setting the data to write
        write_data = []

        body_rows = table_div.find_element(By.TAG_NAME, "tbody")
        body_rows = body_rows.find_elements(By.TAG_NAME, "tr")
        num_rows = len(body_rows) #number of rows
        for row in body_rows:
            curr_row_data = [] #data for the current row
            row_data = row.find_elements(By.TAG_NAME, "td") #find data in these rows
            for i in range(len(row_data)):
                data = row_data[i]
                #if the column is the first one, remove the \n
                if i == 0:  
                    curr_row_data.append(data.text.replace("\n", ": "))
                else: 
                    #remove the percentage sign and convert to a float, if this fails put a 0.0
                    try:
                        curr_row_data.append(float(data.text.replace("%", "")))
                    except:
                        curr_row_data.append(float(0.0))
            write_data.append(curr_row_data) 
        
        #add a row to represent the total percentages
        total_percentages = ["predicted % total vote share"]
        #iterate through rows of the data file, doing a weighted average calculation to add a row for the total estimated percentage
        for i in range(len(write_data)):
            current_row = write_data[i]
            #the first element contains the actual percentage of each row, extract this
            sample_percent = current_row[0].split(":")[1].strip()
            sample_percent = float(sample_percent.replace("%", ""))/100 #remove percentage symbol and divide by 100
            
            #iterate through the rest of the columns
            for j in range(1, len(current_row)):
                current_value = float(current_row[j])
                if i == 0: #for the first row just append the values times the sample percentage
                    total_percentages.append(current_value * sample_percent)
                else: 
                    total_percentages[j] = total_percentages[j] + current_value * sample_percent #else, add the averages
        write_data.append(total_percentages) #append the total percentages row


        #create the dataframe and save the file
        df = pd.DataFrame(write_data, columns=columns)
        path =  "./" + main_path + "/" + area +"_"+str(header) + "_" + str(num_rows) + ".csv" #defining the path as the name of the header, plus the number of rows 
        df.to_csv(path, index=False)

    

A subdirectory or file 2016_presidential_data already exists.


current header:  gender
current header:  age
current header:  age
current header:  age
current header:  race
current header:  race
current header:  area type


In [30]:
#testing reading of exit poll saved csv file
df = pd.read_csv("./2016_presidential_data/wisconsin_area type_3.csv", sep=",")
df.head(n=10)

,sample percentage and category,% for Clinton,% for Trump,% for other/no answer
0,urban area: 39%,61.0,34.0,5.0
1,suburban area: 34%,39.0,55.0,6.0
2,rural area: 27%,35.0,62.0,3.0
3,predicted % total vote share,46.5,48.7,4.8
